In [ ]:
using Pkg
pkg"registry add https://github.com/JuliaRegistries/General"
pkg"registry add https://github.com/vincentcp/FrameFunRegistry"
Pkg.activate(pwd());Pkg.instantiate()

In [ ]:
using LinearAlgebra, DomainSets, BasisFunctions, FrameFun, Plots, PGFPlotsX, Statistics, 
    LaTeXStrings, DocumentPGFPlots
opts = (samplingstyle=OversamplingStyle(), samplingfactor=2, normalizedsampling=true, 
    solverstyle=AZStyle(), threshold=1e-10,REG=pQR_solver)
@pgf sizeopts = {height=140,width=300}
@pgf axisopts = {ymode="log",ymax=1e4,ymin=1e-20,sizeopts...}
@pgf plotopts = {color="black",mark_options="fill=black",mark_size="1pt"}

In [ ]:
D = UnionDomain((-0.75)..(-0.25),0.0..0.5)
P = ExtensionFramePlatform(platform(Fourier(1)→(-1.0..1.0)),D)
N = 50
F = Fun(exp,P,N)
x = PeriodicEquispacedGrid(4N,-1,1)
B = Fourier(N)→(-1.0..1.0)
y = real.(F(x))
yy = real.(Expansion(B,coefficients(F))(x));
m1 = x .∈ Ref(D.domains[1]);
m2 = x .∈ Ref(D.domains[2]);
G = @pgf Axis({cycle_list_name="mark list",
        xlabel=L"N",
        legend_pos="north east",height=140,width=300},
    Plot({no_marks,style="black,dashed,thin"},Table(x,y)),
    Plot({no_marks,style="black,very thick"},Table(x[m1],yy[m1])),
    Plot({no_marks,style="black,very thick"},Table(x[m2],yy[m2])),
)

In [ ]:
DocumentPGFPlots.savefigs(joinpath(pwd(),"FEapprox"), G)

In [ ]:
G = @pgf Axis({cycle_list_name="mark list",
        xlabel=L"N",
        legend_pos="north east",height=140,width=300},
    Plot({only_marks,style="black"},Table(1:N,svdvals(AZ_A(P,N;normalizedsampling=true))))
    )

In [ ]:
DocumentPGFPlots.savefigs(joinpath(pwd(),"FEsvdvals"), G)

# Fourier extension

In [ ]:
P = ExtensionFramePlatform(platform(Fourier(10)→(-1.0..1.0)) ,-.5..0.5)
f = x->x
N = 201

In [ ]:
P

In [ ]:
fA = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),
                svdvals(AZ_A(P, N;opts...,normalizedsampling=false))])))

In [ ]:
fZ = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),
                svdvals(AZ_Zt(P, N;opts..., normalizedsampling=false))])))

In [ ]:
fM = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(plungematrix(P,N;opts...,normalizedsampling=false))])))

In [ ]:
using  Statistics
ns1 = [1<<k for k in 4:16]
ns2 = [1<<k for k in 4:11]
t = [getindex(@timed(approximate(exp,P,n;opts...)),2) for n in ns1,  i in 1:3]
t1 = [getindex(@timed(approximate(exp,P,n;opts...,solverstyle=DirectStyle(),
                    directsolver=:qr)),2) for n in ns2,  i in 1:3]
# p = [plungerank(P,n;opts...) for n in ns];

In [ ]:
fT = @pgf Axis({sizeopts..., xmode="log",ymode="log",legend_pos="south east"},
    PlotInc({plotopts...,mark_size=2,forget_plot},Table([ns1,median(t,dims=2)[:]])),
    PlotInc({plotopts...,mark_size=2,forget_plot},Table([ns2,median(t1,dims=2)[:]])),
    PlotInc({plotopts..., style="black,dashed",mark="none"},
        Table([ns1,1e-5ns1.*log10.(ns1)])),
    LegendEntry(L"\mathcal O(N \log^2(N))"))

In [ ]:
# fP = @pgf Axis({sizeopts...,xmode="log"},PlotInc(plotopts,Table([ns,p])))

In [ ]:
pgfsave("fA.tex" , fA,include_preamble=false)
pgfsave("fZ.tex" , fZ,include_preamble=false)
pgfsave("fM.tex" , fM,include_preamble=false)
pgfsave("fT.tex", fT,include_preamble=false)
# pgfsave("fP.tex", fP,include_preamble=false)

# Legendre extension

In [ ]:
using FrameFun, BasisFunctions
import FrameFun.Platforms: dictionary
import FrameFun.FrameFunInterface: discretemeasure
struct WLegendrePlatform <: BasisPlatform end
FrameFun.Platforms.unsafe_dictionary(plat::WLegendrePlatform, param::Int) = Legendre(param)
FrameFun.Platforms.correctparamformat(platform::WLegendrePlatform, param::Int) = true
discretemeasure(ss::SamplingStyle, plat::WLegendrePlatform, param, ap; options...) =
    gauss_rule(dictionary(plat,length(sampling_grid(ss,ap;options...))))

In [ ]:
P = FrameFun.ExtensionFramePlatform(WLegendrePlatform(), Interval(-.5,.5))
N = 201

In [ ]:
A = AZ_A(P,N; opts...,normalizedsampling=false)

In [ ]:
lA = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(AZ_A(P, N;opts...))])))

In [ ]:
azdual_dict(P,10)

In [ ]:
Z = AZ_Z(P,N; opts...,normalizedsampling=false)

In [ ]:
Zt = AZ_Zt(P,N; opts...,normalizedsampling=false)

In [ ]:
lZ = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(Zt)])))

In [ ]:
lM = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(A-A*Zt*A)])))

In [ ]:
pgfsave("lA.tex" , lA,include_preamble=false)
pgfsave("lZ.tex" , lZ,include_preamble=false)
pgfsave("lM.tex" , lM,include_preamble=false)

In [ ]:
P = FrameFun.OPSExtensionFramePlatform(Legendre(10),Interval(-.5..0.5))
N = 201

In [ ]:
A = AZ_A(P,N; opts...)

In [ ]:
wlA = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(A)])))

In [ ]:
Zt = AZ_Zt(P,N; opts...)

In [ ]:
wlZ = @pgf Axis({axisopts...,ymax=1e4},PlotInc(plotopts,Table([collect(1:N),svdvals(Zt)])))

In [ ]:
wlM = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(A-A*Zt*A)])))

In [ ]:
pgfsave("wlA.tex" , wlA,include_preamble=false)
pgfsave("wlZ.tex" , wlZ,include_preamble=false)
pgfsave("wlM.tex" , wlM,include_preamble=false)

In [ ]:
ns = [1<<k for k in 4:8]
@timed(approximate(exp,P,100;opts...))
@timed(QR_solver(AZ_A(P,100;opts...))*exp)
taz = [getindex(@timed(approximate(exp,P,n;opts...)),2) for n in ns,  i in 1:1]
tqr = [getindex(@timed(QR_solver(AZ_A(P,n;opts...))*exp),2) for n in ns,  i in 1:1]

In [ ]:
wlT = @pgf Axis({xmode="log",ymode="log"},PlotInc(Table([ns,median(taz,dims=2)[:]])),
        LegendEntry("AZ"),
        PlotInc(Table([ns,median(tqr,dims=2)[:]])),
        LegendEntry("QR"))

# Chebyshev extension

In [ ]:
using FrameFun, BasisFunctions
import FrameFun.Platforms: dictionary
import FrameFun.FrameFunInterface: discretemeasure
struct WChebyshevPlatform <: BasisPlatform end
FrameFun.Platforms.unsafe_dictionary(plat::WChebyshevPlatform, param::Int) = ChebyshevT(param)
FrameFun.Platforms.correctparamformat(platform::WChebyshevPlatform, param::Int) = true
discretemeasure(ss::SamplingStyle, plat::WChebyshevPlatform, param, ap; options...) =
    gauss_rule(dictionary(plat,length(sampling_grid(ss,ap;options...))))

In [ ]:
P = FrameFun.ExtensionFramePlatform(WChebyshevPlatform(), Interval(-.5,.5))
N = 201

In [ ]:
A = AZ_A(P,N; opts...)

In [ ]:
cA = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(AZ_A(P, N;opts...))])))

In [ ]:
Zt = AZ_Zt(P,N; opts...)

In [ ]:
cZ = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(AZ_Zt(P, N;opts...))])))

In [ ]:
cM = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:N),svdvals(plungematrix(P,N;opts...))])))

In [ ]:
pgfsave("cA.tex" , cA,include_preamble=false)
pgfsave("cZ.tex" , cZ,include_preamble=false)
pgfsave("cM.tex" , cM,include_preamble=false)

# Weighted on a square

In [ ]:
f = (x,y) -> cos(2pi*(x+y)) + sqrt(x^2+y^2)*sin(1+2pi*(x+y))

In [ ]:
P = WeightedSumPlatform(platform(ChebyshevT(10,-1,1)^2), (x,y)->1., (x,y)->sqrt(x^2+y^2))

In [ ]:
N = 32

In [ ]:
A = AZ_A(P,((N,N),(N,N)); opts...)

In [ ]:
Zt = AZ_Zt(P,((N,N),(N,N)); opts...)

In [ ]:
s = svdvals(A)
wsA = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:length(s)),s])))

In [ ]:
s = svdvals(Zt)
wsZ = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:length(s)),s])))

In [ ]:
s = svdvals(plungematrix(P,((N,N),(N,N));opts...));

In [ ]:
wsM = @pgf Axis(axisopts,
        PlotInc(plotopts,Table([collect(1:length(s)),s])))

In [ ]:
F = Fun(f,P,((N,N),(N,N)); verbose=true,opts...)

In [ ]:
x = EquispacedGrid(100,-1,1)
wsP = plot(F;c=:RdBu,size=(2*300,2*140),layout=2)
wsP = heatmap!(log10.(eps().+abs.(F(x^2)-[f(xi, yi) for xi in x, yi in x]));subplot=2,aspect_ratio=1,ticks=false)

In [ ]:
using Statistics
ns = [1<<k for k in 1:5]
wst = zeros(length(ns),1)
wsNs = [size(AZ_A(P,((n,n),(n,n));opts...),2) for  (i,n) in enumerate(ns)]
@timed(approximate(f,P,((10,10),(10,10));REG=rSVD_solver,opts...))
[wst[i] = getindex(@timed(approximate(f,P,((n,n),(n,n));REG=rSVD_solver,opts...)),2) 
        for (i,n) in enumerate(ns),  j in 1:1]

In [ ]:
wsT = @pgf Axis({sizeopts..., xmode="log",ymode="log",legend_pos="south east"},
        PlotInc(plotopts,Table([ns.^2,median(wst,dims=2)[:]])),
        PlotInc({plotopts..., style="black,dashed",mark="none"},Table([wsNs,1e-5(wsNs).*log.(wsNs)])),
        LegendEntry(L"\mathcal O(N \log(N))"))

In [ ]:
pgfsave("wsA.tex" , wsA,include_preamble=false)
pgfsave("wsZ.tex" , wsZ,include_preamble=false)
pgfsave("wsM.tex" , wsM,include_preamble=false)
pgfsave("wsT.tex" , wsT,include_preamble=false)
Plots.savefig(wsP, "weightedsquare")

# Weighed on a disk

In [ ]:
f = (x,y) -> cos(2pi*(x+y)) + sqrt(x^2+y^2)*sin(1+2pi*(x+y))

In [ ]:
P = FrameFun.ExtensionFramePlatform(WeightedSumPlatform(platform(ChebyshevT(10,-1,1)^2), (x,y)->1., 
        (x,y)->sqrt(x^2+y^2)),.9*UnitDisk())

In [ ]:
N = 32

In [ ]:
A = AZ_A(P,((N,N),(N,N)); opts...)

In [ ]:
Zt = AZ_Zt(P,((N,N),(N,N)); opts...)

In [ ]:
s1 = svdvals(A)

In [ ]:
wcA = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:length(s1)),s1])))

In [ ]:
s2 = svdvals(Zt)

In [ ]:
wcZ = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:length(s2)),s2])))

In [ ]:
s3 = svdvals(plungematrix(P,((N,N),(N,N));opts...))

In [ ]:
wcM = @pgf Axis(axisopts,PlotInc(plotopts,Table([collect(1:length(s3)),s3])))

In [ ]:
F = Fun(f, P, ((N,N),(N,N)); verbose=true,opts...)

In [ ]:
g = PeriodicEquispacedGrid(100,-.9,.9)^2;
y = F(g);

In [ ]:
x = EquispacedGrid(100,-1,1)
wcP = plot(F;c=:RdBu,size=(2*300,2*140),layout=2)
wcP = heatmap!(log10.(eps().+abs.(F(x^2)-[f(xi, yi) for xi in x, yi in x]));subplot=2,aspect_ratio=1,ticks=false)

In [ ]:
using Statistics
ns = [1<<k for k in 1:5]
wct = zeros(length(ns),1)
wcNs = [size(AZ_A(P,((n,n),(n,n));opts...),2) for  (i,n) in enumerate(ns)]
@timed(approximate(f,P,((10,10),(10,10));REG=rSVD_solver,opts...))
[wct[i] = getindex(@timed(approximate(f,P,((n,n),(n,n));REG=rSVD_solver,opts...)),2) 
        for (i,n) in enumerate(ns),  j in 1:1]

In [ ]:
wcT = @pgf Axis({sizeopts..., xmode="log",ymode="log",legend_pos="south east"},
    PlotInc(plotopts,Table([wcNs,median(wct,dims=2)[:]])),
        PlotInc({plotopts..., style="black,dashed",mark="none"},Table([wcNs,1e-5(wcNs).^2])),
        LegendEntry(L"\mathcal O(N^2)"))

In [ ]:
pgfsave("wcA.tex" , wcA,include_preamble=false)
pgfsave("wcZ.tex" , wcZ,include_preamble=false)
pgfsave("wcM.tex" , wcM,include_preamble=false)
pgfsave("wcT.tex" , wcT,include_preamble=false)
savefig(wcP, "weightedcircle")